In [8]:
import sys; sys.path.insert(0, '..')
import utils
import keras
import numpy as np

def embed_and_translate(data, nrows, ncols):
    ndata = np.zeros((len(data), nrows, ncols, 1), dtype='float32')
    translations = np.empty((len(data), 2), dtype='float32')
    width, height = data.shape[1], data.shape[2]
    for i in range(len(data)):
        x = np.random.randint(ncols-width)
        y = np.random.randint(nrows-height)
        ndata[i][x:x+width, y:y+height] = data[i] # rows, cols = height, width
        translations[i][0] = x+(width//2)
        translations[i][1] = y+(height//2)
    return ndata, translations
            
n_splits = 16
io_shape = (64,64,1)
size = int(io_shape[0] // (n_splits**(1/2)))
latent_dim = 16 * 4 # since we have 4x4-splits we need 4 times the latent space for a fair comparison.

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
x_train_augmented, y_train = embed_and_translate(x_train, 64, 64)
x_train_split = np.array([utils.split(x, size, size) for x in x_train_augmented], dtype='float32')

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
x_test_augmented, y_test = embed_and_translate(x_test, 64, 64)
x_test_split = np.array([utils.split(x, size, size) for x in x_test_augmented], dtype='float32')

In [4]:
import tensorflow as tf

model = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(64,64,1), name='enc_in'),
        #tf.keras.layers.Conv2D(filters=int(32), kernel_size=(4,4), strides=(4,4), activation='relu', padding='valid', use_bias=True, name='conv1'),
        tf.keras.layers.Conv2D(filters=int(32), kernel_size=(2,2), strides=(2,2), activation='relu', padding='valid', use_bias=True, name='conv1'),
        tf.keras.layers.Conv2D(filters=int(64), kernel_size=(2,2), strides=(2,2), activation='relu', padding='valid', use_bias=True, name='conv2'),
        tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1), strides=(1,1), activation='sigmoid', padding='valid', name='pt_conv'),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(8,8), strides=(8,8), padding='valid', name='latent_conv', use_bias=True),
        tf.keras.layers.Flatten(name='latent'),
        
        tf.keras.layers.Dense(units=4*8*8*32, activation='relu', name='dense'),
        tf.keras.layers.Reshape(target_shape=(2*8, 2*8, 32), name='reshape'),
        tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='deconv1'),
        tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='deconv2'),
        tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', name='dec_out')
    ]
)
print(model.summary())
model.compile(loss='mse', optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 32)        160       
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 16, 64)        8256      
_________________________________________________________________
pt_conv (Conv2D)             (None, 16, 16, 1)         65        
_________________________________________________________________
latent_conv (Conv2D)         (None, 2, 2, 16)          1040      
_________________________________________________________________
latent (Flatten)             (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 8192)              532480    
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 32)        0

In [5]:
history = model.fit(x_train_augmented, x_train_augmented, validation_data=(x_test_augmented, x_test_augmented), batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0081 - val_loss: 0.0052
Epoch 2/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0048 - val_loss: 0.0045
Epoch 3/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0044 - val_loss: 0.0042
Epoch 4/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0041 - val_loss: 0.0040
Epoch 5/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 6/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 7/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0037 - val_loss: 0.0038
Epoch 8/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0036 - val_loss: 0.0037
Epoch 9/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0036 - val_loss: 0.0036
Epoch 10/10
1875/1875 [==============================] - 28s 15m

In [18]:
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation, Dropout
from keras import Sequential
from keras.models import Model

import tensorflow as tf
import keras
import sys; sys.path.insert(0, '..')
import utils

class MultiSplit(Model):
    def __init__(self, n_splits, latent_dim, io_shape):
        super(MultiSplit, self).__init__()
        n_filters = [32, 64]
        size = int( io_shape[0] // (n_splits**(1/2)) )  # W = H
        self.encoder = self._create_encoder(latent_dim//n_splits, (size, size,1), n_filters)
        self.input_reshaper = utils.Reshaper((n_splits, size, size, 1), (size, size,1))
        self.latent_reshaper = utils.Reshaper([latent_dim//n_splits], [latent_dim])
        self.decoder = self._create_decoder(latent_dim, io_shape, n_filters)
        
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def call(self, x, training=True):
        z = self.encode(x, training=training)
        y_pred = self.decode(z ,training=training)
        return y_pred
        
    def encode(self, x, training=True):
        return self.encoder( self.input_reshaper(x) , training)
    
    def decode(self, z, training=True):
        return self.decoder( self.latent_reshaper(z) , training)
    
    def _create_encoder(self, latent_dim, input_shape, n_filters):
        return Sequential([
            Input(shape=input_shape),
#             Conv2D(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
#             Conv2D(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2D(filters=int(32), kernel_size=(2,2), strides=(2,2), activation='relu', padding='valid', use_bias=True, name='conv1'),
            Conv2D(filters=int(64), kernel_size=(2,2), strides=(2,2), activation='relu', padding='valid', use_bias=True, name='conv2'),
            Conv2D(filters=1, kernel_size=(1,1), strides=(1,1), activation='sigmoid', padding='valid', name='pt_conv'),
#             Conv2D(filters=16, kernel_size=(8,8), strides=(8,8), padding='valid', name='latent_conv', use_bias=True),
            Flatten()
#             Dense(units=latent_dim, activation='sigmoid')  #! Sigmoid activation
        ], name='encoder')

    def _create_decoder(self, latent_dim, io_shape, n_filters):
        return Sequential([
            Input(shape=(latent_dim)),
            Reshape((8,8,1)),
#             Flatten(),
            Conv2D(filters=16, kernel_size=(8,8), strides=(8,8), padding='valid', name='latent_conv', use_bias=True),
#             Dense(io_shape[0]//4 * io_shape[1]//4 * 8),  #! Reduce amount of neurons by 4.
#             Reshape((io_shape[0]//4, io_shape[1]//4, 8)),
#             Conv2DTranspose(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
#             Conv2DTranspose(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
#             Conv2DTranspose(filters=1, kernel_size=(3,3), padding='same'),
#             Activation('sigmoid', name='decoder_out')
            
            Dense(units=4*8*8*32, activation='relu', name='dense'),
            Reshape(target_shape=(2*8, 2*8, 32), name='reshape'),
            Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='deconv1'),
            Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='deconv2'),
            Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', name='dec_out')
        ])

In [19]:
model = MultiSplit(n_splits, latent_dim, io_shape)
model.compile(loss='mse', optimizer='adam') # COMPILE AFTER WEIGHTS LOADED
print(model.encoder.summary())
print(model.decoder.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 8, 8, 32)          160       
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 64)          8256      
_________________________________________________________________
pt_conv (Conv2D)             (None, 4, 4, 1)           65        
_________________________________________________________________
flatten_6 (Flatten)          (None, 16)                0         
Total params: 8,481
Trainable params: 8,481
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 8, 8, 1)           0         
______________________________

In [20]:
history = model.fit(x_train_split, x_train_augmented, validation_data=(x_test_split, x_test_augmented), batch_size=32, epochs=10, shuffle=True)

Epoch 1/10


ValueError: in user code:

    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    <ipython-input-18-a8808b51d140>:24 train_step  *
        loss = self.compiled_loss(y, y_pred)
    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/compile_utils.py:205 __call__  **
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/keras/losses.py:143 __call__
        losses = self.call(y_true, y_pred)
    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/keras/losses.py:246 call
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/keras/losses.py:1198 mean_squared_error
        return K.mean(math_ops.squared_difference(y_pred, y_true), axis=-1)
    /home/don/.local/lib/python3.8/site-packages/tensorflow/python/op